In [1]:
import datetime as dt
import matplotlib.pyplot as plt
from matplotlib import patches
from pathlib import Path
import pandas as pd
import xarray as xr

%matplotlib inline

In [2]:
import geopandas
from shapely.geometry import Polygon

In [3]:
shapefile_dir = Path('/data/MIDOSS/shapefiles/')
vessel_type = 'barge'

## do some logic testing

In [100]:
any([True for x in [3, 4, 5] if x not in [4, 5, 6, 7]])

True

In [98]:
[x for x in [1, 2, 3, 4] if x not in [4, 5, 6, 7]]

[1, 2, 3]

## Back to our Project

In [ ]:
### By number of vessels

In [122]:
def find_alone_associated (innerdata, mmsis, month):
    associated = 0
    alone = 0
    generic = ['Canada', 'US', 'Pacific']
    aug_generic = ['Canada', 'US', 'Pacific', None]
    for ii in mmsis:
        tolist = innerdata[innerdata.MMSI_NUM == ii].TO.unique()
        if any([True for x in tolist if x in generic]):
            if len(tolist) > 4:
                associated = associated + 1
            elif any([True for x in tolist if x not in aug_generic]):
                associate = associated + 1
            else:
                alone = alone + 1
    print (f'Month = {month}. Associated = {associated}, alone = {alone}, ratio alone = {alone/(alone+associated):.2f}')

In [123]:
for i in range(12):
    month = f'{i+1:02d}'
    innerdata = geopandas.read_file(shapefile_dir / 
                        f"{vessel_type}_2018_{month}.shp")
    mmsis = innerdata.MMSI_NUM.unique()
    find_alone_associated(innerdata, mmsis, month)

Month = 01. Associated = 58, alone = 48, ratio alone = 0.45
Month = 02. Associated = 67, alone = 52, ratio alone = 0.44
Month = 03. Associated = 71, alone = 44, ratio alone = 0.38
Month = 04. Associated = 76, alone = 39, ratio alone = 0.34
Month = 05. Associated = 73, alone = 34, ratio alone = 0.32
Month = 06. Associated = 80, alone = 34, ratio alone = 0.30
Month = 07. Associated = 73, alone = 47, ratio alone = 0.39
Month = 08. Associated = 76, alone = 36, ratio alone = 0.32
Month = 09. Associated = 70, alone = 38, ratio alone = 0.35
Month = 10. Associated = 80, alone = 40, ratio alone = 0.33
Month = 11. Associated = 71, alone = 48, ratio alone = 0.40
Month = 12. Associated = 71, alone = 38, ratio alone = 0.35


In [133]:
generic = ['Canada', 'US', 'Pacific']
innerdata[innerdata.TO.isin(generic)]
innerdata[(innerdata.MMSI_NUM == mmsis[5]) & (innerdata.TO.isin(generic))].MMSI_NUM.count()

1382

In [140]:
def count_alone_associated (innerdata, mmsis, month):
    associated = 0
    alone = 0
    generic = ['Canada', 'US', 'Pacific']
    aug_generic = ['Canada', 'US', 'Pacific', None]
    for ii in mmsis:
        tolist = innerdata[innerdata.MMSI_NUM == ii].TO.unique()
        if any([True for x in tolist if x in generic]):
            if len(tolist) > 4:
                associated = associated + innerdata[(innerdata.MMSI_NUM == ii) & (
                                                    ((innerdata.TO.isin(generic)) & (innerdata.FROM_.isin(aug_generic))
                                                     |
                                                     ((innerdata.FROM_.isin(generic)) & (innerdata.TO.isin(aug_generic))
                                                     )
                                                    )
                )].MMSI_NUM.count()
            elif any([True for x in tolist if x not in aug_generic]):
                associate = associated + innerdata[(innerdata.MMSI_NUM == ii) & (
                                                    ((innerdata.TO.isin(generic)) & (innerdata.FROM_.isin(aug_generic))
                                                     |
                                                     ((innerdata.FROM_.isin(generic)) & (innerdata.TO.isin(aug_generic))
                                                     )
                                                    )
                    )].MMSI_NUM.count()
            else:
                alone = alone + innerdata[(innerdata.MMSI_NUM == ii) & (
                                                    ((innerdata.TO.isin(generic)) & (innerdata.FROM_.isin(aug_generic))
                                                     |
                                                     ((innerdata.FROM_.isin(generic)) & (innerdata.TO.isin(aug_generic))
                                                     )
                                                    )
                    )].MMSI_NUM.count()
    print (f'Month = {month}. Associated = {associated}, alone = {alone}, ratio alone = {alone/(alone+associated):.2f}')

In [141]:
count_alone_associated(innerdata, mmsis, month)

Month = 04. Associated = 172016, alone = 171705, ratio alone = 0.50


In [142]:
for i in range(12):
    month = f'{i+1:02d}'
    innerdata = geopandas.read_file(shapefile_dir / 
                        f"{vessel_type}_2018_{month}.shp")
    mmsis = innerdata.MMSI_NUM.unique()
    count_alone_associated(innerdata, mmsis, month)

Month = 01. Associated = 106470, alone = 117671, ratio alone = 0.52
Month = 02. Associated = 171753, alone = 132926, ratio alone = 0.44
Month = 03. Associated = 172016, alone = 171705, ratio alone = 0.50
Month = 04. Associated = 179202, alone = 172528, ratio alone = 0.49
Month = 05. Associated = 194134, alone = 138622, ratio alone = 0.42
Month = 06. Associated = 174730, alone = 123559, ratio alone = 0.41
Month = 07. Associated = 162419, alone = 147476, ratio alone = 0.48
Month = 08. Associated = 172016, alone = 97962, ratio alone = 0.36
Month = 09. Associated = 177927, alone = 157105, ratio alone = 0.47
Month = 10. Associated = 296827, alone = 184810, ratio alone = 0.38
Month = 11. Associated = 274600, alone = 231521, ratio alone = 0.46
Month = 12. Associated = 307405, alone = 199273, ratio alone = 0.39


In [143]:
assoc = [106470, 171753, 172016, 179202, 194134, 174730, 162419, 172016, 177927, 296827, 274600, 307405]
lonely = [117671, 132926, 171705, 172528, 138622, 123559, 147476, 97962, 157105, 184810, 231521, 199273]

In [146]:
print (sum(lonely) / (sum(assoc) + sum(lonely)))

0.4396972605299793


So by number of vessels, about 0.35 are not associated with terminals in a month.  By number of generic tracks, 0.44 are not associated with terminals.